<a href="https://colab.research.google.com/github/UP24Erz/Kressner_Projekt/blob/main/Fallstudie_Elektro.LKW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fallstudie JUICY AG (Network Design)

In [ ]:
! pip install -q pyscipopt

In [ ]:
import pandas as pd
from pyscipopt import Model, quicksum

## Repo & Daten laden, Bsp. Produktions- und Transportkosten

In [ ]:
! git clone https://github.com/AlexKressner/WS25_Supply_Chain_Optimierung

fatal: destination path 'WS25_Supply_Chain_Optimierung' already exists and is not an empty directory.


In [ ]:
! ls

sample_data  WS25_Supply_Chain_Optimierung


In [ ]:
! ls WS25_Supply_Chain_Optimierung/Daten/NetworkDesign/

NetworkDesign_Beschaffungskosten.csv  NetworkDesign_Nachfrage.csv
NetworkDesign_Invest_Kapa.csv	      NetworkDesign_Produktionskosten.csv


In [ ]:
folder = "WS25_Supply_Chain_Optimierung/Daten/NetworkDesign"

In [ ]:
operative_kosten = pd.read_csv(f"{folder}/NetworkDesign_Produktionskosten.csv", sep=";")

In [ ]:
operative_kosten.tail()

,Produktionsstandort,Markt,Produktionskosten
20,East London,USA,142
21,East London,Südamerika,100
22,East London,Europa,103
23,East London,Asien,105
24,East London,Afrika,71


In [ ]:
operative_kosten.head()

,Produktionsstandort,Markt,Produktionskosten
0,Charleston,USA,81
1,Charleston,Südamerika,92
2,Charleston,Europa,101
3,Charleston,Asien,130
4,Charleston,Afrika,115


In [ ]:
kapazitaeten = pd.read_csv(f"{folder}/NetworkDesign_Invest_Kapa.csv", sep=";")

In [ ]:
kapazitaeten.head()

,Produktionsstandort,Ausbaustufe,Kapazitäten,Investitionen
0,Charleston,klein,10,6000
1,Charleston,groß,20,9000
2,Curitiba,klein,10,4500
3,Curitiba,groß,20,6750
4,Hamburg,klein,10,6500


In [ ]:
nachfrage = pd.read_csv(f"{folder}/NetworkDesign_Nachfrage.csv", sep=";")

In [ ]:
nachfrage

,Markt,Nachfragemenge
0,USA,12
1,Südamerika,8
2,Europa,14
3,Asien,16
4,Afrika,7


## Indexmengen

In [ ]:
I = kapazitaeten.Produktionsstandort.unique()
J = list(nachfrage.Markt.unique())
A = kapazitaeten.Ausbaustufe.unique()

## Erweiterung Aufgabe 2

In [ ]:
J.append("Arabische_Halbinsel")

In [ ]:
J

['USA', 'Südamerika', 'Europa', 'Asien', 'Afrika', 'Arabische_Halbinsel']

## Parameter

In [ ]:
# Produktions- und Transportkosten
operative_kosten.set_index(["Produktionsstandort","Markt"], inplace=True)
cp = operative_kosten.to_dict("dict")["Produktionskosten"]

## Erweiterung Aufgabe 2

In [ ]:
cp["Charleston","Arabische_Halbinsel"]=125
cp["Curitiba","Arabische_Halbinsel"]=100
cp["Hamburg","Arabische_Halbinsel"]=100
cp["Quanzhou","Arabische_Halbinsel"]=75
cp["East London","Arabische_Halbinsel"]=85

In [ ]:
# Fixkosten
cf = kapazitaeten.set_index(["Produktionsstandort","Ausbaustufe"])
cf = cf.to_dict("dict")["Investitionen"]

## Erweiterung Aufgabe 1

In [ ]:
cf['Charleston', 'klein'] = cf['Charleston', 'klein'] * 0.7
cf['Charleston', 'groß'] = cf['Charleston', 'groß'] * 0.7

In [ ]:
cf

{('Charleston', 'klein'): 4200.0,
 ('Charleston', 'groß'): 6300.0,
 ('Curitiba', 'klein'): 4500,
 ('Curitiba', 'groß'): 6750,
 ('Hamburg', 'klein'): 6500,
 ('Hamburg', 'groß'): 9750,
 ('Quanzhou', 'klein'): 4100,
 ('Quanzhou', 'groß'): 6150,
 ('East London', 'klein'): 4000,
 ('East London', 'groß'): 6000}

In [ ]:
# Kapazitäten
cap = kapazitaeten.set_index("Ausbaustufe")
cap = cap.to_dict("dict")["Kapazitäten"]
cap

{'klein': 10, 'groß': 20}

In [ ]:
# Nachfrage
nachfrage.set_index("Markt", inplace=True)
d = nachfrage.to_dict("dict")["Nachfragemenge"]
d


{'USA': 12, 'Südamerika': 8, 'Europa': 14, 'Asien': 16, 'Afrika': 7}

## Erweiterung Aufgabe 2

In [ ]:
d["Arabische_Halbinsel"]=5

## Modellinstanz ziehen

In [ ]:
model = Model()

## Entscheidungsvariablen

In [ ]:
Y = {}
for i in I:
  for a in A:
    Y[i,a] = model.addVar(vtype="B", name=f"Ausbaustufe_{a}_Standort_{i}")

In [ ]:
X = {}
for i in I:
  for j in J:
    X[i,j] = model.addVar(vtype="I", name=f"Produktion_Standort_{i}_für_Markt_{j}")

In [ ]:
len(model.getVars())

40

## Zielfunktion

In [ ]:
model.setObjective(
    quicksum(cp[i,j]*X[i,j] for i in I for j in J)
    + quicksum(cf[i,a]*Y[i,a] for i in I for a in A),
    sense="minimize"
    )

## Nebenbedingungen

In [ ]:
# Nachfrage (5)
for j in J:
  model.addCons(quicksum(X[i,j] for i in I) >= d[j], name=f"Nachfrage_Markt_{j}")

In [ ]:
# Kopplung Produktion und Investment (5)
for i in I:
  model.addCons(
      quicksum(X[i,j] for j in J) <= quicksum(cap[a]*Y[i,a] for a in A),
      name=f"Kapazität_Standort_{i}"
      )

In [ ]:
# Nur eine Ausbaustufe (5)
for i in I:
  model.addCons(
      quicksum(Y[i,a] for a in A) <= 1,
      name=f"Ausbaustufe_{i}"
      )

## Erweiterung Aufgabe 3

In [ ]:
# Mindestbeschäftigung
model.addCons(quicksum(X["Charleston",j] for j in J) >= 5, name="Mindestmenge_Charleston")
model.addCons(quicksum(X["Hamburg",j] for j in J) >= 20, name="Mindestmenge_Hamburg")

Mindestmenge_Hamburg

In [ ]:
len(model.getConss())

18

## Lösung

In [ ]:
model.redirectOutput()

In [ ]:
model.optimize()

presolving:
(round 1, fast)       0 del vars, 0 del conss, 0 add conss, 30 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 5 clqs, 0 implints
(round 2, fast)       0 del vars, 0 del conss, 0 add conss, 60 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 5 clqs, 0 implints
(round 3, exhaustive) 0 del vars, 0 del conss, 0 add conss, 60 chg bounds, 0 chg sides, 0 chg coeffs, 5 upgd conss, 0 impls, 5 clqs, 0 implints
   (0.0s) probing cycle finished: starting next cycle
   (0.0s) symmetry computation started: requiring (bin +, int +, cont +), (fixed: bin -, int -, cont -)
   (0.0s) no symmetry present (symcode time: 0.00)
presolving (4 rounds: 4 fast, 2 medium, 2 exhaustive):
 0 deleted vars, 0 deleted constraints, 0 added constraints, 60 tightened bounds, 0 added holes, 0 changed sides, 0 changed coefficients
 36 implications, 5 cliques, 0 implied integral variables (0 bin, 0 int, 0 cont)
presolved problem has 40 variables (10 bin, 30 int, 0 cont) and 18 constra

In [ ]:
print('LÖSUNG:')
print('Zielfunktionswert (Kosten) =', model.getObjVal())
print("\n")
print("PRODUKTIONS-MARKT-ZUTEILUNG")
for j in J:
  print(f"{j}:")
  for i in I:
    if model.getVal(X[i,j])>0:
      print('\t', f'{i,j} =', round(model.getVal(X[i,j])))
print("\n")
print("INVESTITIONEN")
for i in I:
  for a in A:
    if model.getVal(Y[i,a])>0:
      print(f'{i,a} =', round(model.getVal(Y[i,a])))

LÖSUNG:
Zielfunktionswert (Kosten) = 31090.0


PRODUKTIONS-MARKT-ZUTEILUNG
USA:
	 ('Charleston', 'USA') = 10
	 ('Hamburg', 'USA') = 2
Südamerika:
	 ('Hamburg', 'Südamerika') = 4
	 ('East London', 'Südamerika') = 4
Europa:
	 ('Hamburg', 'Europa') = 14
Asien:
	 ('Quanzhou', 'Asien') = 16
Afrika:
	 ('East London', 'Afrika') = 7
Arabische_Halbinsel:
	 ('Quanzhou', 'Arabische_Halbinsel') = 4
	 ('East London', 'Arabische_Halbinsel') = 1


INVESTITIONEN
('Charleston', 'klein') = 1
('Hamburg', 'groß') = 1
('Quanzhou', 'groß') = 1
('East London', 'groß') = 1


In [ ]:
model.getStatus()

'optimal'